In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd


url = "https://www.ptt.cc/bbs/C_Chat/index.html"

for i in range(50): 
 
    r = requests.get(url)
    soup = BeautifulSoup(r.text,"html.parser")
    rents = soup.find_all("div","r-ent")
    u = soup.select("div.btn-group.btn-group-paging a")
    url = "https://www.ptt.cc"+ u[1]["href"]  #上一頁的網址

    titles = [r.find('div',"title").text.strip() for r in rents]
    dates = [d.find('div',"meta").find("div","date").text.strip() for d in rents]

    #存成dataframe
    df = pd.DataFrame()  #暫存當頁資料

    df=pd.DataFrame(
    {
        '標題': titles,
        "日期": dates,
    })
    

In [3]:
df.head(5)

,標題,日期


In [6]:
import jieba
import jieba.analyse
import pandas as pd
import numpy as np
from collections import Counter

In [7]:
df['seg'] = " "
df.head()

,標題,日期,seg
0,[閒聊] 雷電將軍那麼火是不是代表奶大就夠了?,11/15,
1,[閒聊] 戰神怎那麼快就被提名入圍TGA？,11/15,
2,[閒聊] 比古的斬魄刀會是什麼樣子？,11/15,
3,[閒聊] 統神神預言「ALL IN台積電」全家爆買20張,11/15,
4,[閒聊] 如果要讓小智退場，應該怎麼做比較好?,11/15,


In [8]:
jieba.analyse.set_stop_words('stopwords.txt')

index = 0
for documents in df["標題"]:
    seg_list = jieba.analyse.extract_tags(documents, 150)
    seg_list = jieba.lcut(documents)
    df['seg'].iloc[index] = seg_list
    index += 1

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\babe9\AppData\Local\Temp\jieba.cache
Loading model cost 0.590 seconds.
Prefix dict has been built successfully.


In [9]:
df["seg"]

0     [[, 閒聊, ],  , 雷電將, 軍, 那麼, 火, 是不是, 代表, 奶大, 就夠, ...
1     [[, 閒聊, ],  , 戰神, 怎, 那麼, 快, 就, 被, 提名, 入圍, TGA, ？]
2            [[, 閒聊, ],  , 比古, 的, 斬魄, 刀會, 是, 什麼, 樣子, ？]
3     [[, 閒聊, ],  , 統神神, 預言, 「, ALL,  , IN, 台積電, 」, ...
4     [[, 閒聊, ],  , 如果, 要, 讓, 小智, 退場, ，, 應該, 怎麼, 做, ...
5              [[, Vtub, ],  , 原來, 阿火還, 沒, 破百, 萬, =, =]
6           [[, 閒聊, ],  , 戰神, 遊戲, 銷售, 有, 辦法, 超越, 法環, 嗎]
7     [[, 新聞, ], MILE, 推出, 大選遊戲,  , 辨識, 假資訊, 政治, 操弄,...
8                [[, 閒聊, ],  , 中岡, ：, 你們, 當然會, 看, 我, 吧]
9     [Re, :,  , [, 閒聊, ],  , 墊, 肩能, 幹, 嘛, ?,  , (, ...
10         [[, 閒聊, ],  , 坂田銀時, 的, 斬魄, 刀會, 是, 什麼, 樣子, ？]
11    [Re, :,  , [, 閒聊, ],  , 戰神, 遊戲, 銷售, 有, 辦法, 超越,...
12                [(, 本文, 已, 被, 刪除, ),  , [, Barrel, ]]
13    [Re, :,  , [, 閒聊, ],  , 小數值, 遊戲, 比, 較, 好, 還是, ...
14          [(, 本文, 已, 被, 刪除, ),  , [, JustBecauseU, ]]
15                  [[, 閒聊, ],  , FX, 戰士, 韭菜, 芽,  , 19]
16            [[, 閒聊, ],  , 友, 哈哈, 戰戀次, 到底, 去, 幹, 嘛, 的]
17                       [[, 奶子, ],  , 小孤獨, 掌握, 

In [10]:
u = 0
words = []
for i in df['seg']:
    for word in df["seg"][u]:
        words.append(word)
    u = u+1

In [11]:
show_key = pd.DataFrame(Counter(words).keys())
show_values = pd.DataFrame(Counter(words).values())

In [12]:
df1 = [show_key, show_values]
df1

[       0
 0      [
 1     閒聊
 2      ]
 3       
 4    雷電將
 ..   ...
 131   神皮
 132  雙人組
 133    ！
 134   超低
 135   傷害
 
 [136 rows x 1 columns],
       0
 0    20
 1    13
 2    20
 3    27
 4     1
 ..   ..
 131   1
 132   1
 133   1
 134   1
 135   1
 
 [136 rows x 1 columns]]

In [13]:
result = pd.concat(df1, axis=1)
result.columns = ["有提到的單字", "次數"]
result

,有提到的單字,次數
0,[,20
1,閒聊,13
2,],20
3,,27
4,雷電將,1
...,...,...
131,神皮,1
132,雙人組,1
133,！,1
134,超低,1


In [18]:
import plotly.express as px
px.bar(result, x="有提到的單字",y="次數",title="title")

ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

Figure({
    'data': [{'alignmentgroup': 'True',
              'hovertemplate': '有提到的單字=%{x}<br>次數=%{y}<extra></extra>',
              'legendgroup': '',
              'marker': {'color': '#636efa', 'pattern': {'shape': ''}},
              'name': '',
              'offsetgroup': '',
              'orientation': 'v',
              'showlegend': False,
              'textposition': 'auto',
              'type': 'bar',
              'x': array(['[', '閒聊', ']', ' ', '雷電將', '軍', '那麼', '火', '是不是', '代表', '奶大', '就夠',
                          '了', '?', '戰神', '怎', '快', '就', '被', '提名', '入圍', 'TGA', '？', '比古', '的',
                          '斬魄', '刀會', '是', '什麼', '樣子', '統神神', '預言', '「', 'ALL', 'IN', '台積電', '」',
                          '全家', '爆買', '20', '張', '如果', '要', '讓', '小智', '退場', '，', '應該', '怎麼', '做',
                          '比', '較', '好', 'Vtub', '原來', '阿火還', '沒', '破百', '萬', '=', '遊戲', '銷售',
                          '有', '辦法', '超越', '法環', '嗎', '新聞', 'MILE', '推出', '大選遊戲', '辨識', '假資訊',
                          '政治', '操弄', '手段', '中岡', '：', '你們', '當然會', '看', '我', '吧', 'Re', ':', '墊',
                          '肩能', '幹', '嘛', '(', 'EX', '.', '七龍珠', '比克', '衣服', '那種', ')', '坂田銀時',
                          '本文', '已', '刪除', 'Barrel', '小數值', '還是', '大數值', 'JustBecauseU', 'FX',
                          '戰士', '韭菜', '芽', '19', '友', '哈哈', '戰戀次', '到底', '去', '奶子', '小孤獨', '掌握',
                          '流量', '密碼', '水星', '第七', '話', '先行', '圖', '並', '不是', '玩家', '赤手空拳', '打敗',
                          '神皮', '雙人組', '！', '超低', '傷害'], dtype=object),
              'xaxis': 'x',
              'y': array([20, 13, 20, 27,  1,  1,  2,  1,  1,  1,  1,  1,  1,  3,  3,  1,  1,  1,
                           3,  1,  1,  1,  4,  1,  4,  2,  2,  2,  2,  2,  1,  1,  2,  1,  1,  1,
                           2,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  2,  2,  2,  1,
                           1,  1,  1,  1,  1,  2,  3,  2,  2,  2,  2,  3,  2,  1,  1,  1,  1,  1,
                           1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  3,  3,  1,  1,  2,  2,  4,
                           1,  1,  1,  1,  1,  1,  3,  1,  2,  2,  2,  1,  1,  1,  1,  1,  1,  1,
                           1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
                           1,  1,  1,  1,  1,  1,  1,  1,  1,  1], dtype=int64),
              'yaxis': 'y'}],
    'layout': {'barmode': 'relative',
               'legend': {'tracegroupgap': 0},
               'template': '...',
               'title': {'text': 'title'},
               'xaxis': {'anchor': 'y', 'domain': [0.0, 1.0], 'title': {'text': '有提到的單字'}},
               'yaxis': {'anchor': 'x', 'domain': [0.0, 1.0], 'title': {'text': '次數'}}}
})

In [17]:
from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import WordCloud
#mask = np.array(Image.open("bg.jpg"))
font = "C:\Windows\Fonts\MSJH.TTC"
wordcloud = WordCloud(background_color = "white", font_path=font)
wordcloud.generate_from_frequencies(result)
plt.figure(figsize=(20,20))
plt.imshow(wordcloud)
plt.axis("off")
plt.show()

TypeError: '<' not supported between instances of 'str' and 'int'